In [ ]:
  !pip install yfinance
import pandas as pd
import yfinance as yf # import libraries
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore') 
from google.colab import files
import tqdm.notebook as tq

     |████████████████████████████████| 6.3 MB 18.7 MB/s 
  Created wheel for yfinance: filename=yfinance-0.1.63-py2.py3-none-any.whl size=23919 sha256=59d1bc33c5add4bc46c130907f409e9e8937080599be48b430ffa6484e8b1daf
  Stored in directory: /root/.cache/pip/wheels/fe/87/8b/7ec24486e001d3926537f5f7801f57a74d181be25b11157983
Successfully built yfinance
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6


In [ ]:
nse_Symbol=['ADANIPORTS.NS',
 'ASIANPAINT.NS',
 'AXISBANK.NS',
 'BAJAJ-AUTO.NS',
 'BAJAJFINSV.NS',
 'BAJFINANCE.NS',
 'BHARTIARTL.NS',
 'BPCL.NS',
 'BRITANNIA.NS',
 'CIPLA.NS',
 'COALINDIA.NS',
 'DIVISLAB.NS',
 'DRREDDY.NS',
 'EICHERMOT.NS',
 'GRASIM.NS',
 'HCLTECH.NS',
 'HDFC.NS',
 'HDFCBANK.NS',
 'HDFCLIFE.NS',
 'HEROMOTOCO.NS',
 'HINDALCO.NS',
 'HINDUNILVR.NS',
 'ICICIBANK.NS',
 'INDUSINDBK.NS',
 'INFY.NS',
 'IOC.NS',
 'ITC.NS',
 'JSWSTEEL.NS',
 'KOTAKBANK.NS',
 'LT.NS',
 'M&M.NS',
 'MARUTI.NS',
 'NESTLEIND.NS',
 'NTPC.NS',
 'ONGC.NS',
 'POWERGRID.NS',
 'RELIANCE.NS',
 'SBILIFE.NS',
 'SBIN.NS',
 'SHREECEM.NS',
 'SUNPHARMA.NS',
 'TATACONSUM.NS',
 'TATAMOTORS.NS',
 'TATASTEEL.NS',
 'TCS.NS',
 'TECHM.NS',
 'TITAN.NS',
 'ULTRACEMCO.NS',
 'UPL.NS',
 'WIPRO.NS']

# TO CHECK

In [ ]:

# nifty
df1 = yf.download('^NSEI', 
                  start='2021-07-14', 
                  end='2021-07-16,
                  interval='5m')
df1.reset_index(inplace=True)
df1['Datetime']=df1['Datetime'].astype('str')
df1['Points']= df1['Close'].diff()
df1['Points']=[df1['Points'][i] if df1['Open'][i] < df1['Close'][i] else (0-df1['Points'][i]) for i in range(0,len(df1))]
df1=df1[df1['Datetime'].str.contains('09:15')]
df1=df1.reset_index(drop=True)

# stocks
df2 = yf.download('ACC.NS', 
                  start='2021-06-14', 
                  end='2021-07-16',
                  interval='5m')
df2.reset_index(inplace=True)
df2['Datetime']=df2['Datetime'].astype('str')
df2=df2[df2['Datetime'].str.contains('09:15')]
df2=df2.reset_index(drop=True)

df2['Nifty']=df1['Points']
df2['Selection']=['Yes' if df2['Open'][i] == df2['Low'][i] and df2['Nifty'][i] > 0 else 'No' for i in range(0,len(df2))] 
df2=df2[df2['Selection'] == 'Yes'].reset_index(drop=True)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


# NEGATIVE

In [ ]:
def checker(stock_name,start_date,end_date): 
  # nifty
  df1 = yf.download('^NSEI', 
                    start=start_date, 
                    end=end_date,
                    interval='5m')
  df1.reset_index(inplace=True)
  df1['Datetime']=df1['Datetime'].astype('str')
  df1['Points']= df1['Close'].diff()
  df1=df1[df1['Datetime'].str.contains('09:15')]
  df1=df1.reset_index(drop=True)
  # stocks
  df2 = yf.download(stock_name, 
                    start=start_date, 
                    end=end_date,
                    interval='5m')
  df2.reset_index(inplace=True)
  df2['Datetime']=df2['Datetime'].astype('str')
  df2=df2[df2['Datetime'].str.contains('09:15')]
  df2=df2.reset_index(drop=True)
  df2['Nifty']=df1['Points']
  df2['Height'] = df2['Open'] - df2['Close'].shift(1)
  # df2['Selection']=['Yes' if df2['Open'][i] == df2['Low'][i] and df2['Nifty'][i] > 0 and df2['Height'][i] > 0 else 'No' for i in range(0,len(df2))] 
  df2['Selection']=['Yes' if df2['Open'][i] == df2['High'][i] and df2['Nifty'][i] < 0 and df2['Height'][i] < 0 else 'No' for i in range(0,len(df2))] 
  df2=df2[df2['Selection'] == 'Yes'].reset_index(drop=True)
  return (list(df2['Close']),list(df2['Datetime']))

selected_stocks={}
for i in tq.tqdm(nse_Symbol):
    selected_stocks[i]=checker(i,'2021-07-10','2021-07-20')
final_selection=pd.DataFrame.from_dict(selected_stocks,orient='index',columns=['Close','Dates'])
final_selection.reset_index(inplace=True)
final_selection=final_selection.rename(columns={'index':'Ticker'})
final_selection=final_selection.reset_index().drop('index',axis=1)
# add all in proper list format
new_col=[]
for i in tq.tqdm(range(0,len(final_selection))):
  new_entry=[]
  for j in range(0,len(final_selection['Dates'][i])):
    new_entry.append([final_selection['Ticker'][i],final_selection['Dates'][i][j],final_selection['Close'][i][j]])
  new_col.append(new_entry)
# make it to proper csv file
final = pd.DataFrame(columns=['Stock','Date','Close'])
for i in tq.tqdm(range(0,len(new_col))):
  for j in new_col[i]:
    try:
      final = final.append({'Stock': j[0],'Date':j[1],'Close':j[2]}, ignore_index=True)
    except:
      pass
# print('completed, please check dataframe "final" ')
final=final.dropna().reset_index(drop=True)
final['Date']=final['Date'].astype('str')
final['Shares']=  round(65500/final['Close'] )
# final['Stock']='NSE:' + final['Stock'].str.split('.',expand=True)[0]
final['Date']=final['Date'].str.split(' ',expand=True)[0]
final=final.sort_values(by='Date').reset_index(drop=True)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [ ]:
final

,Stock,Date,Close,Shares
0,ASIANPAINT.NS,2021-07-14,2988.850098,22.0
1,BAJAJ-AUTO.NS,2021-07-14,3952.899902,17.0
2,EICHERMOT.NS,2021-07-14,2696.050049,24.0
3,SHREECEM.NS,2021-07-14,27947.000000,2.0


In [ ]:
# final.to_excel('Intraday.xlsx',index=False)
# files.download('Intraday.xlsx')

In [ ]:
# rm 'Intraday.xlsx'

# POSITIVE

In [ ]:
def checker(stock_name,start_date,end_date): 
  # nifty
  df1 = yf.download('^NSEI', 
                    start=start_date, 
                    end=end_date,
                    interval='5m')
  df1.reset_index(inplace=True)
  df1['Datetime']=df1['Datetime'].astype('str')
  df1['Points']= df1['Open'] - df1['Close'].shift(1)
  df1=df1[df1['Datetime'].str.contains('09:15')]
  df1=df1.reset_index(drop=True)
  # stocks
  df2 = yf.download(stock_name, 
                    start=start_date, 
                    end=end_date,
                    interval='5m')
  df2.reset_index(inplace=True)
  df2['Datetime']=df2['Datetime'].astype('str')
  df2=df2[df2['Datetime'].str.contains('09:15')]
  df2=df2.reset_index(drop=True)
  df2['Nifty']=df1['Points']
  df2['Height'] = df2['Open'] - df2['Close'].shift(1)
  df2['Selection']=['Yes' if df2['Open'][i] == df2['Low'][i] and df2['Nifty'][i] > 0 and df2['Height'][i] > 0 else 'No' for i in range(0,len(df2))] 
  df2=df2[df2['Selection'] == 'Yes'].reset_index(drop=True)
  return (list(df2['Close']),list(df2['Datetime']))

selected_stocks={}
for i in tq.tqdm(nse_Symbol):
    selected_stocks[i]=checker(i,'2021-07-19','2021-07-24')
final_selection=pd.DataFrame.from_dict(selected_stocks,orient='index',columns=['Close','Dates'])
final_selection.reset_index(inplace=True)
final_selection=final_selection.rename(columns={'index':'Ticker'})
final_selection=final_selection.reset_index().drop('index',axis=1)
# add all in proper list format
new_col=[]
for i in tq.tqdm(range(0,len(final_selection))):
  new_entry=[]
  for j in range(0,len(final_selection['Dates'][i])):
    new_entry.append([final_selection['Ticker'][i],final_selection['Dates'][i][j],final_selection['Close'][i][j]])
  new_col.append(new_entry)
# make it to proper csv file
final = pd.DataFrame(columns=['Stock','Date','Close'])
for i in tq.tqdm(range(0,len(new_col))):
  for j in new_col[i]:
    try:
      final = final.append({'Stock': j[0],'Date':j[1],'Close':j[2]}, ignore_index=True)
    except:
      pass
# print('completed, please check dataframe "final" ')
final=final.dropna().reset_index(drop=True)
final['Date']=final['Date'].astype('str')
final['Shares']=  round(65500/final['Close'] )


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [ ]:
# final['Stock']='NSE:' + final['Stock'].str.split('.',expand=True)[0]
# final['Date']=final['Date'].str.split(' ',expand=True)[0]
final=final.sort_values(by='Date').reset_index(drop=True)

In [ ]:
# final.to_excel('Pos_Intraday.xlsx',index=False)
# files.download('Pos_Intraday.xlsx')

In [ ]:
# rm 'Pos_Intraday.xlsx'

In [ ]:
final

,Stock,Date,Close,Shares
0,HCLTECH.NS,2021-07-23 09:15:00+05:30,987.950012,66.0
1,MARUTI.NS,2021-07-23 09:15:00+05:30,7350.000000,9.0
2,ONGC.NS,2021-07-23 09:15:00+05:30,116.500000,562.0
3,SBILIFE.NS,2021-07-23 09:15:00+05:30,1032.949951,63.0
4,WIPRO.NS,2021-07-23 09:15:00+05:30,587.950012,111.0


In [ ]:
# ADD NEW HIGH ALONG WITH OP=LOW with volume greater than previous day
# ADD NEW LOW ALONG WITH OP=HIgh with volume greater than previous day